In [ ]:
def movieId_by_urserId(userid,df):
    """
    Input
    -----
        userid:int
        df:dataframe of train/test rating csv
    Return
    -----
    a list of movie id that rated by given user"""
    return list(df[df['userId'] == userid]['movieId'])

In [ ]:
def search_rating_user_id(userid,movieid):
    """
    Input
    ---
    userid:int, represent userid
    movieid:int, represent movieid (closest id that rated by given user)"""
    return rating[(rating['userId'] == userid) & (rating['movieId'] == movieid)]['rating']

In [ ]:
def cosine_similarity (test_movie_id,userid):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        user_id: user id,int
    Return
    
    ----
        """
    #############
    #list of movie id for given user in train,int
    train_movieid_given_user = movieId_by_urserId(userid,data_train)
    #list of movie id for given user in train,str
    train_movieid_given_user_str = [str(val) for val in train_movieid_given_user]
    
    
    ###########
    
    
    l_latent_movie = []
    movie_cos_dic = {}
    for movie in train_movieid_given_user_str:
        #print(movie,type(movie)) #2，str

        numerator =array([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[movie]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)# cosimilarity value for test latent vs all train latent
        
        dict_movieid_cos = dict(zip(train_movieid_given_user_str,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1))
        
    return [ele for ele in reversed(l_sorted_s_to_l)] 